In [1]:
import sys, time
import pickle
import sklearn.preprocessing
from scipy import stats
import pandas as pd
import numpy as np
from _plotly_future_ import v4_subplots
import plotly.graph_objs as go
import plotly.io as pio
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.figure_factory as ff
plotly.io.orca.config.executable = '/anaconda2/envs/pytorch/bin/orca'
init_notebook_mode(connected=True)

glob_layout = go.Layout(
    font=dict(family='Helvetica', size=24, color='black'),
    margin=dict(l=100, r=10, t=10, b=100),
    xaxis=dict(showgrid=False,  zeroline=False, ticks="inside", showline=True,
               tickwidth=3, linewidth=3, ticklen=10,
               mirror="allticks", color="black"),
    yaxis=dict(showgrid=False,  zeroline=False, ticks="inside", showline=True,
               tickwidth=3, linewidth=3, ticklen=10,
               mirror="allticks", color="black"),
    legend_orientation="v",
)

In [2]:
from perato_utils import is_pareto, area_under_pareto
from plot_utils import *
from gpr import process_generation_2DEI, gp_predict
from ei import getEiVec2D_aug, getPiVec2D_aug, get_ei_samples_kmedoids

/anaconda2/envs/pytorch/lib/python3.6/site-packages/matplotlib/style/core.py:167: UserWarning:

In /Users/duanchenru/.matplotlib/stylelib/myline.mplstyle: Duplicate key in file "/Users/duanchenru/.matplotlib/stylelib/myline.mplstyle", line #10



In [3]:
fnames = pickle.load(open('fnames.pkl', 'rb'))
df = pd.read_json('df_RAC155_homogap.json', orient='records', lines=True)
whole_percentage = 100
df = df.sample(n=int(whole_percentage*0.01*len(df)), random_state=0)
df = df.reset_index()

In [4]:
y1l = 'alphaHOMO'
y2l = 'gap'

In [5]:
pareto_inds, pareto_points = is_pareto(points=df[[y1l, y2l]].values)

In [6]:
pi_pareto_area_list = pickle.load(open("./2DEI_alphaHOMO_gap/demo/100percent/pi_kmedoids/start2_initpoints_300_totgen_20_points_50_from_500/pareto_area_list.pkl", "rb"))
pi_coverage_list = pickle.load(open("./2DEI_alphaHOMO_gap/demo/100percent/pi_kmedoids/start2_initpoints_300_totgen_20_points_50_from_500/coverage_list.pkl", "rb"))
pi_pareto_list = pickle.load(open("./2DEI_alphaHOMO_gap/demo/100percent/pi_kmedoids/start2_initpoints_300_totgen_20_points_50_from_500/pareto_list.pkl", "rb"))
pi_df = pickle.load(open("./2DEI_alphaHOMO_gap/demo/100percent/pi_kmedoids/start2_initpoints_300_totgen_20_points_50_from_500/df.pkl", "rb"))
pi_pareto_inds_pred, pi_pareto_points_pred = is_pareto(points=pi_df[['hat_y1', 'hat_y2']].values)
pi_pareto_pred_actual_space = pi_df.iloc[pi_pareto_inds_pred][[y1l, y2l]].values
pi_df_rm1stfront = pi_df[~pi_df['unique_name'].isin(pi_df.iloc[pi_pareto_inds_pred]['unique_name'].values)]
pi_pareto_inds_pred_2nd, pi_pareto_points_pred_2nd = is_pareto(
        points=pi_df_rm1stfront[['hat_y1', 'hat_y2']].values)
pi_pareto_pred_actual_space_2nd = pi_df_rm1stfront.iloc[pi_pareto_inds_pred_2nd][[y1l, y2l]].values

In [7]:
ei_pareto_area_list = pickle.load(open("./2DEI_alphaHOMO_gap/demo/100percent/ei_kmedoids/start2_initpoints_300_totgen_20_points_50_from_500/pareto_area_list.pkl", "rb"))
ei_coverage_list = pickle.load(open("./2DEI_alphaHOMO_gap/demo/100percent/ei_kmedoids/start2_initpoints_300_totgen_20_points_50_from_500/coverage_list.pkl", "rb"))
ei_pareto_list = pickle.load(open("./2DEI_alphaHOMO_gap/demo/100percent/ei_kmedoids/start2_initpoints_300_totgen_20_points_50_from_500/pareto_list.pkl", "rb"))
ei_df = pickle.load(open("./2DEI_alphaHOMO_gap/demo/100percent/ei_kmedoids/start2_initpoints_300_totgen_20_points_50_from_500/df.pkl", "rb"))
ei_pareto_inds_pred, ei_pareto_points_pred = is_pareto(points=ei_df[['hat_y1', 'hat_y2']].values)
ei_pareto_pred_actual_space = ei_df.iloc[ei_pareto_inds_pred][[y1l, y2l]].values
ei_df_rm1stfront = ei_df[~ei_df['unique_name'].isin(ei_df.iloc[ei_pareto_inds_pred]['unique_name'].values)]
ei_pareto_inds_pred_2nd, ei_pareto_points_pred_2nd = is_pareto(
        points=ei_df_rm1stfront[['hat_y1', 'hat_y2']].values)
ei_pareto_pred_actual_space_2nd = ei_df_rm1stfront.iloc[ei_pareto_inds_pred_2nd][[y1l, y2l]].values

In [8]:
dpf_pareto_area_list = pickle.load(open("./2DEI_alphaHOMO_gap/demo/100percent/PFSampling/start2_initpoints_300_totgen_100_points_10/pareto_area_list.pkl", "rb"))
dpf_coverage_list = pickle.load(open("./2DEI_alphaHOMO_gap/demo/100percent/PFSampling/start2_initpoints_300_totgen_100_points_10/coverage_list.pkl", "rb"))
dpf_pareto_list = pickle.load(open("./2DEI_alphaHOMO_gap/demo/100percent/PFSampling/start2_initpoints_300_totgen_100_points_10/pareto_list.pkl", "rb"))
dpf_df = pickle.load(open("./2DEI_alphaHOMO_gap/demo/100percent/PFSampling/start2_initpoints_300_totgen_100_points_10/df.pkl", "rb"))
dpf_pareto_inds_pred, dpf_pareto_points_pred = is_pareto(points=dpf_df[['hat_y1', 'hat_y2']].values)
dpf_pareto_pred_actual_space = dpf_df.iloc[dpf_pareto_inds_pred][[y1l, y2l]].values
dpf_df_rm1stfront = dpf_df[~dpf_df['unique_name'].isin(dpf_df.iloc[dpf_pareto_inds_pred]['unique_name'].values)]
dpf_pareto_inds_pred_2nd, dpf_pareto_points_pred_2nd = is_pareto(
        points=dpf_df_rm1stfront[['hat_y1', 'hat_y2']].values)
dpf_pareto_pred_actual_space_2nd = dpf_df_rm1stfront.iloc[dpf_pareto_inds_pred_2nd][[y1l, y2l]].values

In [9]:
dpfpi_pareto_area_list = pickle.load(open("./2DEI_alphaHOMO_gap/demo/100percent/dpf_pi_kmedoids/start2_initpoints_300_totgen_50_points_10_from_100/pareto_area_list.pkl", "rb"))
dpfpi_coverage_list = pickle.load(open("./2DEI_alphaHOMO_gap/demo/100percent/dpf_pi_kmedoids/start2_initpoints_300_totgen_50_points_10_from_100/coverage_list.pkl", "rb"))
dpfpi_pareto_list = pickle.load(open("./2DEI_alphaHOMO_gap/demo/100percent/dpf_pi_kmedoids/start2_initpoints_300_totgen_50_points_10_from_100/pareto_list.pkl", "rb"))
dpfpi_df = pickle.load(open("./2DEI_alphaHOMO_gap/demo/100percent/dpf_pi_kmedoids/start2_initpoints_300_totgen_50_points_10_from_100/df.pkl", "rb"))
dpfpi_pareto_inds_pred, dpfpi_pareto_points_pred = is_pareto(points=dpfpi_df[['hat_y1', 'hat_y2']].values)
dpfpi_pareto_pred_actual_space = dpfpi_df.iloc[dpfpi_pareto_inds_pred][[y1l, y2l]].values
dpfpi_df_rm1stfront = dpfpi_df[~dpfpi_df['unique_name'].isin(dpfpi_df.iloc[dpf_pareto_inds_pred]['unique_name'].values)]
dpfpi_pareto_inds_pred_2nd, dpfpi_pareto_points_pred_2nd = is_pareto(
        points=dpfpi_df_rm1stfront[['hat_y1', 'hat_y2']].values)
dpfpi_pareto_pred_actual_space_2nd = dpfpi_df_rm1stfront.iloc[dpfpi_pareto_inds_pred_2nd][[y1l, y2l]].values

In [10]:
random_pareto_area_list = pickle.load(open("./2DEI_alphaHOMO_gap/demo/100percent/random/start1_initpoints_150_totgen_100_points_150/pareto_area_list.pkl", "rb"))
random_coverage_list = pickle.load(open("./2DEI_alphaHOMO_gap/demo/100percent/random/start1_initpoints_150_totgen_100_points_150/coverage_list.pkl", "rb"))

In [11]:
trace0 = go.Scatter(
        x=pi_coverage_list,
        y=pi_pareto_area_list,
        mode='markers+lines',
        opacity=1,
        name = "pi",
        marker=dict(
            size=5,
            color='blue'
        ),
    )
trace1 = go.Scatter(
        x=random_coverage_list,
        y=random_pareto_area_list,
        mode='markers+lines',
        name = "random",
        opacity=1,
        marker=dict(
            size=5,
            color='black'
        ),
    )
data = [trace0, trace1]
layout = go.Layout()
layout.update(glob_layout)
layout["xaxis"].update({'title': "coverage", "type": "log"})
layout["yaxis"].update({'title': "Pareto area"})
layout.legend.update(x=0.05, y=1.0, bgcolor='rgba(0,0,0,0)')
layout.update(xaxis=dict(), yaxis=dict())
layout.update(height=500, width=500, showlegend=True)
fig = dict(data=data, layout=layout)
iplot(fig)

In [12]:
trace0 = go.Scatter(
    x=pi_coverage_list,
    y=pi_pareto_area_list,
    mode='markers+lines',
    name='pi',
    opacity=1,
    marker=dict(
        size=5,
        color='blue'
    ),
)
trace1 = go.Scatter(
    x=ei_coverage_list,
    y=ei_pareto_area_list,
    mode='markers+lines',
    name='ei',
    opacity=1,
    marker=dict(
        size=5,
        color='red'
    ),
)
trace2 = go.Scatter(
    x=dpf_coverage_list,
    y=dpf_pareto_area_list,
    mode='markers+lines',
    name='pf',
    opacity=1,
    marker=dict(
        size=5,
        color='green'
    ),
)
trace3 = go.Scatter(
    x=dpfpi_coverage_list,
    y=dpfpi_pareto_area_list,
    mode='markers+lines',
    name='pi+pf',
    opacity=1,
    marker=dict(
        size=5,
        color='orange'
    ),
)
data = [trace0, trace1, trace2, trace3]
layout = go.Layout()
layout.update(glob_layout)
layout["xaxis"].update({'title': "coverage"})
layout["yaxis"].update({'title': "Pareto area"})
layout.legend.update(x=0.7, y=0.1, bgcolor='rgba(0,0,0,0)')
layout.update(xaxis=dict(), yaxis=dict())
layout.update(height=500, width=500, showlegend=True)
fig = dict(data=data, layout=layout)
iplot(fig)

In [ ]:
trace0 = go.Scatter(
    x=pi_pareto_list[-1][:, 0],
    y=pi_pareto_list[-1][:, 1],
    mode='markers',
    opacity=1,
    name='pi',
    marker=dict(
        size=8,
        color='blue',
        symbol='circle',
    ),
)
trace1 = go.Scatter(
    x=ei_pareto_list[-1][:, 0],
    y=ei_pareto_list[-1][:, 1],
    mode='markers',
    opacity=1,
    name='ei',
    marker=dict(
        size=8,
        color='red',
        symbol='circle',
    ),
)
trace2 = go.Scatter(
    x=dpf_pareto_list[-1][:, 0],
    y=dpf_pareto_list[-1][:, 1],
    mode='markers',
    name='pi+pf',
    opacity=1,
    marker=dict(
        size=8,
        color='green',
        symbol='square-open',
        line={"width": 2}
    ),
)
trace3 = go.Scatter(
    x=dpfpi_pareto_list[-1][:, 0],
    y=dpfpi_pareto_list[-1][:, 1],
    mode='markers',
    name='pf',
    opacity=1,
    marker=dict(
        size=8,
        color='orange',
        symbol='diamond-open',
        line={"width": 2}
    ),
)
data = [trace0, trace1, trace2, trace3]
layout = go.Layout()
layout.update(glob_layout)
layout["xaxis"].update({'title': "alphaHOMO"})
layout["yaxis"].update({'title': "gap"})
layout.legend.update(x=0.6, y=1.0, bgcolor='rgba(0,0,0,0)')
layout.update(xaxis=dict(), yaxis=dict())
layout.update(height=500, width=500, showlegend=True)
fig = dict(data=data, layout=layout)
iplot(fig)

In [ ]:
_, pi_pareto_pred_actual_space_pareto = is_pareto(points=pi_pareto_pred_actual_space)
_, ei_pareto_pred_actual_space_pareto = is_pareto(points=ei_pareto_pred_actual_space)
_, dpf_pareto_pred_actual_space_pareto = is_pareto(points=dpf_pareto_pred_actual_space)
_, dpfpi_pareto_pred_actual_space_pareto = is_pareto(points=dpfpi_pareto_pred_actual_space)

In [ ]:
trace0 = go.Scatter(
    x=pi_pareto_pred_actual_space_pareto[:, 0],
    y=pi_pareto_pred_actual_space_pareto[:, 1],
    mode='markers',
    opacity=1,
    name='pi',
    marker=dict(
        size=8,
        color='blue',
        symbol='circle',
    ),
)
trace1 = go.Scatter(
    x=ei_pareto_pred_actual_space_pareto[:, 0],
    y=ei_pareto_pred_actual_space_pareto[:, 1],
    mode='markers',
    opacity=1,
    name='ei',
    marker=dict(
        size=8,
        color='red',
        symbol='circle',
    ),
)
trace2 = go.Scatter(
    x=dpf_pareto_pred_actual_space_pareto[:, 0],
    y=dpf_pareto_pred_actual_space_pareto[:, 1],
    mode='markers',
    name='pf',
    opacity=1,
    marker=dict(
        size=8,
        color='green',
        symbol='square-open',
        line={"width": 2}
    ),
)
trace3 = go.Scatter(
    x=dpfpi_pareto_pred_actual_space_pareto[:, 0],
    y=dpfpi_pareto_pred_actual_space_pareto[:, 1],
    mode='markers',
    name='pi+pf',
    opacity=1,
    marker=dict(
        size=8,
        color='orange',
        symbol='square-open',
        line={"width": 2}
    ),
)
data = [trace0, trace1, trace2, trace3]
layout = go.Layout()
layout.update(glob_layout)
layout["xaxis"].update({'title': "alphaHOMO"})
layout["yaxis"].update({'title': "gap"})
layout.legend.update(x=0.6, y=1.0, bgcolor='rgba(0,0,0,0)')
layout.update(xaxis=dict(), yaxis=dict())
layout.update(height=500, width=500, showlegend=True)
fig = dict(data=data, layout=layout)
iplot(fig)